In [1]:
import requests
import json
import uuid
import time
signalStation = "http://biohack.personalhealthtrain.net"

def getStations(signalStation):
    response = requests.get(signalStation + "/client")
    return json.loads(response.text)

def defineTrain(image, inputVars, runId):
    return {
        "image": image,
        "inputString": ("%s" % json.dumps(inputVars)),
        "runId": runId
    }

def requestExecution(signalStation, clientId, taskDescription):
    response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskDescription)
    taskId = json.loads(response.text)["taskId"]
    return taskId

def getExecutionResult(signalStation, clientId, taskId, wait=True):
    url = signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result"
    jsonResult = json.loads(requests.get(url).text)
    
    while len(jsonResult)==0 & wait:
        time.sleep(2)
        jsonResult = json.loads(requests.get(url).text)
    
    return jsonResult

In [2]:
getStations(signalStation)

[{'country': 'The Netherlands',
  'email': 'j.vansoest@maastrichtuniversity.nl',
  'id': 1,
  'institute': 'Maastricht University',
  'ip': '192.168.176.1',
  'last_seen': '2019-09-02 19:14:13.829209',
  'name': 'Trusted Secure Environment (TSE)'},
 {'country': 'The Netherlands',
  'email': 'j.vansoest@maastrichtuniversity.nl',
  'id': 2,
  'institute': 'Maastricht University',
  'ip': '192.168.176.1',
  'last_seen': '2019-09-02 19:14:17.864963',
  'name': 'Data Party A'},
 {'country': 'The Netherlands',
  'email': 'j.vansoest@maastrichtuniversity.nl',
  'id': 3,
  'institute': 'Maastricht University',
  'ip': '192.168.176.1',
  'last_seen': '2019-09-02 19:14:14.689868',
  'name': 'Data Party B'}]

**********************

## Data Party 1 ##

In [36]:
inputData = {
    "party_name": "party_1", # unique name
    "data_file": "https://raw.githubusercontent.com/sunchang0124/BioHackathon2019/localRunning/containers/createContainer/Party_4_Container/data_party_4.csv", # path of the data 
    "salt_text": "apple", # the agreed on salt (all data parties use the same salt)
    "id_feature": ["housenum", "zipcode", "date_of_birth", "sex"] # personal identifier features used for linking purpose
}

myTrain = defineTrain("sophia921025/dataparty:15th", inputData, 1)


In [37]:
clientId = 2
taskId = requestExecution(signalStation, clientId, myTrain)
print("Task submitted with ID: %s" % taskId)

Task submitted with ID: 65


In [38]:
results = getExecutionResult(signalStation, clientId, taskId)
# print(results[0])
result = json.loads(results[0]["response"])
print(result)

{'party_1fileUUID': '9bbdaa6d-10a7-4d44-a943-89ab42deaf90', 'party_1encryptKey': 'QmDg6JGJAi5uNkKfRFldTFPuYeIrL2naqHkFaeeFkBo=', 'party_1verifyKey': 'WuO+Oet26ltHZLr9d99ShB/vZpPA3pGYNnbPOvjlW9A='}


******************

## Data Party 2 ##

In [39]:
inputData = {
    "party_name": "party_4", # unique name
    "data_file": "https://raw.githubusercontent.com/sunchang0124/BioHackathon2019/localRunning/containers/createContainer/Party_4_Container/data_party_4.csv", # path of the data 
    "salt_text": "apple", # the agreed on salt (all data parties use the same salt)
    "id_feature": ["housenum", "zipcode", "date_of_birth", "sex"] # personal identifier features used for linking purpose
}

myTrain = defineTrain("sophia921025/dataparty:15th", inputData, 1)


In [40]:
clientId = 3
taskId = requestExecution(signalStation, clientId, myTrain)
print("Task submitted with ID: %s" % taskId)

Task submitted with ID: 66


In [41]:
results = getExecutionResult(signalStation, clientId, taskId)
# print(results[0])
result = json.loads(results[0]["response"])
print(result)

{'party_4fileUUID': 'f8f88836-fd10-4566-ba7a-935feb75b819', 'party_4encryptKey': 'BLwFvl5HJTWFJj1roMW6ojIzx/HtDd5F9VkYCBbx5hQ=', 'party_4verifyKey': 'LZhE/FWpMOD5nT2fxQe+RDeTMqX+WQNJxAxNxRskWpc='}


## AT TSE ##

In [42]:
### AT TSE ###
inputData = {"parties": ["party_1","party_4"],
        "party_1fileUUID": "6ce2807b-65b2-4068-9315-271015e71a02", 
        "party_1encryptKey": "nlC8gQHZhDFudS1FFZ/l0URoC2Xd1/zOSpB96S1bwQc=", 
        "party_1verifyKey": "ylYWKO2ZIcJKxNi7XNp0BhZ94icmrGOt0QmSMrEglqk=",
        "party_4fileUUID": "8d219572-8f43-4454-a55e-c81bcdee4960", 
        "party_4encryptKey": "cucAlsBrvnqjKs5MXwcmafODcJA1pAV2N5iPArlE+5w=", 
        "party_4verifyKey": "nZmwYQb3RW2660AV1Q3M6lS3XNFYHbBPRmJJUm8MzrI="}

myTrain = defineTrain("sophia921025/tse:4thtry", inputData, 1)

In [43]:
clientId = 1
taskId = requestExecution(signalStation, clientId, myTrain)
print("Task submitted with ID: %s" % taskId)

Task submitted with ID: 67


In [45]:
results = getExecutionResult(signalStation, clientId, taskId)
print(results[0]["response"])
# result = json.loads(results[0]["response"])
# print(result)

MSE train data: 1106.017, MSE test data: 1052.849
R2 train data: 0.957, R2 test data: 0.957
